# DATA 304 – Module 6, Session 1

## Relational Databases with SQLite: Import, Query, Join, Aggregate
**Goal:** Load data into SQLite, practice SQL queries, joins, and aggregation from Python.

**What you'll learn:**
- Create and connect to a SQLite database
- Import CSV data into tables
- Run SELECT, WHERE, ORDER BY, GROUP BY, HAVING
- Perform INNER and LEFT JOINs
- Create indexes and use parameterized queries
- Export results back to CSV

**Prereqs:** Python 3, `pandas`

## 1) Setup

In [ ]:
import sqlite3, os
import pandas as pd
print(pd.__version__)

## 2) Create SQLite DB and tables

In [ ]:
db_path = './data/module6_session1.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS employees;')
cur.execute('DROP TABLE IF EXISTS departments;')

cur.execute('''CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT NOT NULL
);''')

cur.execute('''CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name   TEXT NOT NULL,
    dept_id INTEGER,
    salary REAL,
    hire_date TEXT,
    FOREIGN KEY(dept_id) REFERENCES departments(dept_id)
);''')
conn.commit()
print('Tables created.')

## 3) Load CSVs into tables

In [ ]:
df_emp = pd.read_csv('data/employees.csv')
df_dep = pd.read_csv('data/departments.csv')

df_dep.to_sql('departments', conn, if_exists='append', index=False) #if_exists → 'fail' (default, error if table exists), 'replace' (drop & recreate), 'append' (insert rows).
df_emp.to_sql('employees', conn, if_exists='append', index=False)

pd.read_sql('SELECT * FROM employees LIMIT 5;', conn)

## 4) Basic SELECT, WHERE, ORDER BY

In [ ]:
pd.read_sql(''' 
                SELECT emp_id, name, salary
                FROM employees
                ORDER BY salary DESC;
            ''', conn)

In [ ]:
pd.read_sql(''' 
                SELECT name, salary 
                FROM employees 
                WHERE salary >= 80000 
                ORDER BY salary DESC;
            ''', conn)

## 5) Aggregation with GROUP BY and HAVING

In [ ]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                GROUP BY dept_id
                ORDER BY avg_salary DESC;
            ''', conn)

In [ ]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                GROUP BY dept_id
                HAVING AVG(salary) >= 80000;
            ''', conn)

## 6) Joins

In [ ]:
pd.read_sql('''
                SELECT e.emp_id, e.name, d.dept_name, e.salary
                FROM employees e
                INNER JOIN departments d ON e.dept_id = d.dept_id
                ORDER BY e.emp_id;
            ''', conn)

In [ ]:
pd.read_sql('''
                SELECT d.dept_id, d.dept_name, e.emp_id, e.name
                FROM departments d
                LEFT JOIN employees e ON e.dept_id = d.dept_id
                ORDER BY d.dept_id;
            ''', conn)

## 7) Parameterized queries

In [ ]:
min_salary = 80000
sql = 'SELECT name, salary FROM employees WHERE salary >= ? ORDER BY salary DESC;'
pd.read_sql(sql, conn, params=(min_salary,))

In [ ]:
min_salary = 80000
max_salary = 100000
sql = 'SELECT name, salary FROM employees WHERE salary >= ? AND salary < ? ORDER BY salary DESC;'
pd.read_sql(sql, conn, params=(min_salary, max_salary))

## 8) Indexes and EXPLAIN QUERY PLAN

In [ ]:
pd.read_sql('EXPLAIN QUERY PLAN SELECT name FROM employees WHERE salary >= 80000;', conn)

In [ ]:
cur.execute('CREATE INDEX IF NOT EXISTS idx_emp_salary ON employees(salary);')
conn.commit()
pd.read_sql('EXPLAIN QUERY PLAN SELECT name FROM employees WHERE salary >= 80000;', conn)

## 9) Export results

In [ ]:
res = pd.read_sql('SELECT name, salary FROM employees WHERE salary >= 80000;', conn)
res.to_csv('data/high_paid.csv', index=False)
res.head()

## 10) Mini-exercises

1. List employees hired in or after 2021

2. Total payroll per department

3. Departments with zero employees

## 11) Close connection

In [ ]:
conn.close()